In [42]:
import calendar
from datetime import date
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# Weekday names to calendar constants
WEEKDAYS = {day: getattr(calendar, day.upper()) for day in ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]}

# Bangladesh public holidays (fixed Gregorian dates)
HOLIDAYS = {
    (1, 1): "New Year's Day", (2, 21): "International Mother Language Day", (3, 17): "Sheikh Mujibur Rahman's Birthday",
    (3, 26): "Independence Day", (4, 14): "Bengali New Year", (5, 1): "May Day", (5, 9): "Buddha Purnima",
    (6, 16): "Eid-ul-Azha", (6, 28): "Eid-ul-Azha Holiday", (7, 7): "Eid-e-Milad-un-Nabi", (8, 15): "National Mourning Day",
    (9, 6): "Ashura", (10, 1): "Durga Puja (Vijaya Dashami)", (10, 24): "Eid-ul-Fitr", (12, 16): "Victory Day", (12, 25): "Christmas Day"
}

# Input widgets
year_slider = widgets.IntSlider(value=2025, min=1900, max=2100, description='Year:', layout=widgets.Layout(width='50%'))
weekday_dropdown = widgets.Dropdown(options=list(WEEKDAYS.keys()), value='Sunday', description='Start Day:', layout=widgets.Layout(width='50%'))
view_toggle = widgets.ToggleButtons(options=['Year', 'Month'], description='View:', button_style='info')
month_dropdown = widgets.Dropdown(options=[(calendar.month_name[i], i) for i in range(1, 13)], description='Month:', layout=widgets.Layout(width='50%'))
date_picker = widgets.DatePicker(description='Pick a Date:', layout=widgets.Layout(width='50%'))

# Output areas
calendar_output = widgets.Output()
holiday_output = widgets.Output()

# Create calendar text with holidays
def generate_calendar(year, start_day, mode, month=None):
    cal = calendar.TextCalendar(WEEKDAYS[start_day])
    if mode == 'Year':
        return cal.formatyear(year, 2, 1, 8, 3)
    text = cal.formatmonth(year, month, 8, 2)
    holidays = [f"🎉 {calendar.month_name[month]} {day}: {name}" for (m, day), name in HOLIDAYS.items() if m == month]
    return text + ("\nHolidays:\n" + "\n".join(holidays) if holidays else '')

# Show selected date's holiday if exists
def show_holiday_info(change):
    with holiday_output:
        clear_output()
        d = date_picker.value
        if d:
            key = (d.month, d.day)
            if key in HOLIDAYS:
                msg = f"🎉 {d.strftime('%B %d')}: {HOLIDAYS[key]}"
                color = 'green'
            else:
                msg = f"📅 {d.strftime('%B %d')} is not a holiday."
                color = 'gray'
            display(HTML(f"<b style='color:{color};'>{msg}</b>"))

# Display calendar
def display_calendar(change=None):
    with calendar_output:
        clear_output()
        y, start, mode = year_slider.value, weekday_dropdown.value, view_toggle.value
        m = month_dropdown.value if mode == 'Month' else None
        title = f"📅 Calendar for {calendar.month_name[m]} {y}" if m else f"📅 Calendar for {y}"
        print(title)
        print("=" * 40)
        print(generate_calendar(y, start, mode, m))

# Event bindings
year_slider.observe(display_calendar, 'value')
weekday_dropdown.observe(display_calendar, 'value')
view_toggle.observe(display_calendar, 'value')
month_dropdown.observe(display_calendar, 'value')
date_picker.observe(show_holiday_info, 'value')

# Layout and display
ui = widgets.VBox([
    widgets.HTML("<h2 style='color: teal;'>📅 Bangladesh Holiday Calendar</h2>"),
    year_slider, weekday_dropdown, view_toggle, month_dropdown,
    date_picker, holiday_output, calendar_output
])

display(ui)
display_calendar()